In [ ]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda,RunnableSequence
from langchain_ollama.llms import OllamaLLM

load_dotenv()

model = OllamaLLM(model='llama3.2:latest')

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的文字摘要助手，擅長將長篇文章濃縮成簡潔的重點摘要。"),
        ("human", "請將以下文章摘要成 {max_length} 字以內的重點：\n\n{article}"),
    ]
)

format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))

invoke_model = RunnableLambda(lambda x:model.invoke(x.to_messages()))

parse_output = RunnableLambda(lambda x:x.content)


In [13]:
def add_debug_log(step_name):
    def _log(x):
        print(f"\n{'='*60}")
        print(f"[DEBUG] 步驟:{step_name}")
        print(f"[DEBUG] 輸入的類型:{type(x).__name__}")
        if isinstance(x,str):
            print(f"[DEBUG] 內容預覽:{x[:100]}..." if len(x)>100 else f"[DEBUG] 內容預覽:{x}")
        print(f"{'='*60}")

    return RunnableLambda(_log)

In [ ]:
chain = RunnableSequence(
    first=add_debug_log("開始處理"),
    middle = [format_prompt,
            add_debug_log("格式化完成"),
            invoke_model,
            add_debug_log("模型調用完成"),
            ],
    last= parse_output
    )
sample_article = """
人工智慧（AI）是電腦科學的一個分支，旨在創建能夠執行通常需要人類智能的任務的系統。
這些任務包括學習、推理、問題解決、感知和語言理解。AI 技術已經在許多領域產生重大影響，
包括醫療保健、金融、交通運輸和娛樂。機器學習是 AI 的一個子領域，它使計算機能夠從數據中學習
而無需明確編程。深度學習是機器學習的一個分支，使用人工神經網絡來模擬人腦的工作方式。
隨著技術的進步，AI 有望在未來幾年繼續改變我們的生活和工作方式。
"""
response = chain.invoke({
    "article":sample_article,
    "max_length":100
    })
response


[DEBUG] 步驟:開始處理
[DEBUG] 輸入的類型:dict


TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'NoneType'>.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT 